#  __CAPSTONE PROJECT: IBM DATA SCIENCE PROFESSIONAL COURSE__

## __1. Problem statement-__

#### Family who is living in Charlotte, NC and planning to open a restaurant. Before they think about other things require to open a restaurant, they would like to do assessment on different area based on area, nearby restaurant etc. This will help them comapre and decide best location. 

## __2.Description of Data__

Based on the problem that we are trying to solve, assessment on following list based on zip code would be important.

1. Zip code list of the charlotte
2. Near by Restaurant

We will use

1. https://docs.gaslamp.media/download-zip-code-latitude-longitude-city-state-county-csv/ - To get list of zipcode in charlotte area
2. Number of restaurants and their type and location in every neighborhood will be obtained using Foursquare API

# 3. Methodology

In this section we will be doing following steps

1. Collect Inspection Data
2. Explore and Understand Data
3. Data preparation and preprocessing 
4. Modeling

# 1. Collect Inspection Data

## Import library


In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
import urllib.request # library to open URLs
from bs4 import BeautifulSoup
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# import k-means from clustering stage
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                       

## 2. Explore and understand data

### Read .CSV file --> we will use http://docs.gaslamp.media/wp-content/uploads/2013/08/zip_codes_states.csv to get list of zipcode and area name in charlotte.

This .csv file contain every zip code with its City, State, County name along with Latitude and Longitude.

In [2]:
url='http://docs.gaslamp.media/wp-content/uploads/2013/08/zip_codes_states.csv'
D=pd.read_csv(url)
D.head()

,zip_code,latitude,longitude,city,state,county
0,501,40.922326,-72.637078,Holtsville,NY,Suffolk
1,544,40.922326,-72.637078,Holtsville,NY,Suffolk
2,601,18.165273,-66.722583,Adjuntas,PR,Adjuntas
3,602,18.393103,-67.180953,Aguada,PR,Aguada
4,603,18.455913,-67.145780,Aguadilla,PR,Aguadilla


## 3. Data preparing and Data processing 

1. extract all rows for city name 'Charlotte'
2. Filter all rows for state name 'NC'

In [3]:
Charlotte_data=D[D['city'].str.contains('Charlotte')].reset_index(drop=True)
Charlotte_data.head()

,zip_code,latitude,longitude,city,state,county
0,5445,44.316580,-73.226034,Charlotte,VT,Chittenden
1,12036,42.541025,-74.655395,Charlotteville,NY,Schoharie
2,20622,38.484421,-76.754943,Charlotte Hall,MD,Saint Marys
3,22901,38.093604,-78.561139,Charlottesville,VA,Albemarle
4,22902,38.026588,-78.480450,Charlottesville,VA,Charlottesville City


In [4]:
Charlotte_data=Charlotte_data[Charlotte_data['state'].str.contains('NC')].reset_index(drop=True)
Charlotte_data.head()

,zip_code,latitude,longitude,city,state,county
0,28201,35.260020,-80.804151,Charlotte,NC,Mecklenburg
1,28202,35.226192,-80.843493,Charlotte,NC,Mecklenburg
2,28203,35.208992,-80.856343,Charlotte,NC,Mecklenburg
3,28204,35.215342,-80.828706,Charlotte,NC,Mecklenburg
4,28205,35.325959,-80.827444,Charlotte,NC,Mecklenburg


In [5]:
CLIENT_ID = 'Q5UTMZ53L2KMCAAHVAT20IL3USY5LGZ30E5ANU5LUKNUAH2G' # your Foursquare ID
CLIENT_SECRET = '420JAA1GXTWMVZONJ4DX4OOAUOYSYW2Z2UHBSO5HJKBTNKWF' # your Foursquare Secret
VERSION = '20200605'
LIMIT = 100
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: Q5UTMZ53L2KMCAAHVAT20IL3USY5LGZ30E5ANU5LUKNUAH2G
CLIENT_SECRET:420JAA1GXTWMVZONJ4DX4OOAUOYSYW2Z2UHBSO5HJKBTNKWF


In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            radius, 
            LIMIT)
                
        # make the GET request
        results = requests.get(url).json()['response']['groups'][0]['items']
        # assign relevant part of JSON to venues
        #print(results)
               
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
      
    #for v in results
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [7]:
Charlotte_venues = getNearbyVenues(names='Charlotte',
                                   latitudes=Charlotte_data['latitude'],
                                   longitudes=Charlotte_data['longitude']
                                  )

In [8]:
print(Charlotte_venues.shape)
Charlotte_venues.head()

(385, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,C,35.26002,-80.804151,Popeyes Louisiana Kitchen,35.256585,-80.797171,Fried Chicken Joint
1,C,35.26002,-80.804151,Budget Car Rental,35.266466,-80.807570,Rental Car Location
2,C,35.26002,-80.804151,United States Postal Service,35.253929,-80.801192,Post Office
3,C,35.26002,-80.804151,7-Eleven,35.255222,-80.801427,Convenience Store
4,C,35.26002,-80.804151,Hart Witzen Gallery,35.251608,-80.807855,Art Gallery


In [9]:
Charlotte_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
C,13,13,13,13,13,13
a,92,92,92,92,92,92
e,47,47,47,47,47,47
h,100,100,100,100,100,100
l,3,3,3,3,3,3
o,8,8,8,8,8,8
r,80,80,80,80,80,80
t,42,42,42,42,42,42


In [10]:
# one hot encoding
Charlotte_onehot = pd.get_dummies(Charlotte_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Charlotte_onehot['Neighborhood'] = Charlotte_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Charlotte_onehot.columns[-1]] + list(Charlotte_onehot.columns[:-1])
Charlotte_onehot = Charlotte_onehot[fixed_columns]

Charlotte_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,BBQ Joint,...,Tapas Restaurant,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
Charlotte_grouped = Charlotte_onehot.groupby('Neighborhood').mean().reset_index()
Charlotte_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Terminal,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Tapas Restaurant,Thai Restaurant,Theater,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Women's Store
0,C,0.000000,0.0,0.0,0.0,0.000000,0.076923,0.00,0.000000,0.000000,...,0.00000,0.000000,0.00,0.00000,0.000000,0.0,0.076923,0.000000,0.000000,0.00000
1,a,0.010870,0.0,0.0,0.0,0.032609,0.000000,0.00,0.000000,0.021739,...,0.01087,0.021739,0.00,0.01087,0.000000,0.0,0.000000,0.021739,0.010870,0.01087
2,e,0.042553,0.0,0.0,0.0,0.042553,0.000000,0.00,0.021277,0.000000,...,0.00000,0.000000,0.00,0.00000,0.021277,0.0,0.000000,0.000000,0.021277,0.00000
3,h,0.000000,0.0,0.0,0.0,0.040000,0.010000,0.01,0.000000,0.000000,...,0.01000,0.010000,0.04,0.00000,0.000000,0.0,0.000000,0.000000,0.010000,0.00000
4,l,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.00,0.000000,0.000000,...,0.00000,0.000000,0.00,0.00000,0.000000,0.0,0.000000,0.000000,0.000000,0.00000


In [12]:
kclusters = 5

Charlotte_grouped_clustering = Charlotte_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Charlotte_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 1, 1, 1, 0, 2, 1, 4], dtype=int32)

In [13]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [14]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Charlotte_grouped['Neighborhood']

for ind in np.arange(Charlotte_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Charlotte_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,C,Rental Car Location,Taco Place,Convenience Store,Seafood Restaurant,Fried Chicken Joint,Bank,Grocery Store,Spanish Restaurant,Post Office,Art Gallery
1,a,Coffee Shop,Pizza Place,Brewery,American Restaurant,Furniture / Home Store,Mexican Restaurant,Bar,Gourmet Shop,Restaurant,Gym / Fitness Center
2,e,Yoga Studio,Bakery,Food Truck,Liquor Store,Mexican Restaurant,Deli / Bodega,Coffee Shop,Pizza Place,Burger Joint,Pub
3,h,Hotel,Steakhouse,Restaurant,Park,Pizza Place,American Restaurant,Theater,Sandwich Place,Coffee Shop,Lounge
4,l,Gym / Fitness Center,Breakfast Spot,Discount Store,Farmers Market,Fried Chicken Joint,French Restaurant,Food Truck,Financial or Legal Service,Fast Food Restaurant,Women's Store
5,o,American Restaurant,Convenience Store,Fast Food Restaurant,Brewery,Food Truck,Discount Store,Burger Joint,Fried Chicken Joint,French Restaurant,Financial or Legal Service
6,r,Pizza Place,Sandwich Place,New American Restaurant,Coffee Shop,American Restaurant,Park,Ice Cream Shop,Bar,Chinese Restaurant,Mexican Restaurant
7,t,American Restaurant,Spa,Intersection,French Restaurant,Coffee Shop,Gym / Fitness Center,Italian Restaurant,Gym,Café,Sculpture Garden


In [15]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Charlotte_merged = Charlotte_venues

# merge charlotte_grouped with charlotte_data to add latitude/longitude for each neighborhood
Charlotte_merged = Charlotte_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Charlotte_merged.head() # check the last columns!

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,C,35.26002,-80.804151,Popeyes Louisiana Kitchen,35.256585,-80.797171,Fried Chicken Joint,3,Rental Car Location,Taco Place,Convenience Store,Seafood Restaurant,Fried Chicken Joint,Bank,Grocery Store,Spanish Restaurant,Post Office,Art Gallery
1,C,35.26002,-80.804151,Budget Car Rental,35.266466,-80.807570,Rental Car Location,3,Rental Car Location,Taco Place,Convenience Store,Seafood Restaurant,Fried Chicken Joint,Bank,Grocery Store,Spanish Restaurant,Post Office,Art Gallery
2,C,35.26002,-80.804151,United States Postal Service,35.253929,-80.801192,Post Office,3,Rental Car Location,Taco Place,Convenience Store,Seafood Restaurant,Fried Chicken Joint,Bank,Grocery Store,Spanish Restaurant,Post Office,Art Gallery
3,C,35.26002,-80.804151,7-Eleven,35.255222,-80.801427,Convenience Store,3,Rental Car Location,Taco Place,Convenience Store,Seafood Restaurant,Fried Chicken Joint,Bank,Grocery Store,Spanish Restaurant,Post Office,Art Gallery
4,C,35.26002,-80.804151,Hart Witzen Gallery,35.251608,-80.807855,Art Gallery,3,Rental Car Location,Taco Place,Convenience Store,Seafood Restaurant,Fried Chicken Joint,Bank,Grocery Store,Spanish Restaurant,Post Office,Art Gallery


In [16]:
# create map
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

latitude=35.2271
longitude=80.8431
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
#rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Charlotte_merged['Neighborhood Latitude'], Charlotte_merged['Neighborhood Longitude'], Charlotte_merged['Neighborhood'], Charlotte_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        #color=rainbow[cluster-1],
        fill=True,
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [67]:
Charlotte_merged.loc[Charlotte_merged['Cluster Labels'] == 0, Charlotte_merged.columns[[1] + list(range(5, Charlotte_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
282,35.325959,-80.826335,Breakfast Spot,0,Discount Store,Gym / Fitness Center,Breakfast Spot,Women's Store,Fast Food Restaurant,Farmers Market,Dry Cleaner,Donut Shop,Doctor's Office,Diner
283,35.325959,-80.826528,Discount Store,0,Discount Store,Gym / Fitness Center,Breakfast Spot,Women's Store,Fast Food Restaurant,Farmers Market,Dry Cleaner,Donut Shop,Doctor's Office,Diner
284,35.325959,-80.828493,Gym / Fitness Center,0,Discount Store,Gym / Fitness Center,Breakfast Spot,Women's Store,Fast Food Restaurant,Farmers Market,Dry Cleaner,Donut Shop,Doctor's Office,Diner


In [68]:
Charlotte_merged.loc[Charlotte_merged['Cluster Labels'] == 1, Charlotte_merged.columns[[1] + list(range(5, Charlotte_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,35.226192,-80.841951,Performing Arts Venue,1,Hotel,Steakhouse,Restaurant,Theater,Pizza Place,Park,American Restaurant,Pub,Coffee Shop,Lounge
12,35.226192,-80.841663,Concert Hall,1,Hotel,Steakhouse,Restaurant,Theater,Pizza Place,Park,American Restaurant,Pub,Coffee Shop,Lounge
13,35.226192,-80.841747,Hotel Bar,1,Hotel,Steakhouse,Restaurant,Theater,Pizza Place,Park,American Restaurant,Pub,Coffee Shop,Lounge
14,35.226192,-80.844124,Italian Restaurant,1,Hotel,Steakhouse,Restaurant,Theater,Pizza Place,Park,American Restaurant,Pub,Coffee Shop,Lounge
15,35.226192,-80.841974,American Restaurant,1,Hotel,Steakhouse,Restaurant,Theater,Pizza Place,Park,American Restaurant,Pub,Coffee Shop,Lounge
16,35.226192,-80.845154,Steakhouse,1,Hotel,Steakhouse,Restaurant,Theater,Pizza Place,Park,American Restaurant,Pub,Coffee Shop,Lounge
17,35.226192,-80.843737,Steakhouse,1,Hotel,Steakhouse,Restaurant,Theater,Pizza Place,Park,American Restaurant,Pub,Coffee Shop,Lounge
18,35.226192,-80.844526,Gym,1,Hotel,Steakhouse,Restaurant,Theater,Pizza Place,Park,American Restaurant,Pub,Coffee Shop,Lounge
19,35.226192,-80.845046,Bakery,1,Hotel,Steakhouse,Restaurant,Theater,Pizza Place,Park,American Restaurant,Pub,Coffee Shop,Lounge
20,35.226192,-80.846126,Café,1,Hotel,Steakhouse,Restaurant,Theater,Pizza Place,Park,American Restaurant,Pub,Coffee Shop,Lounge


In [69]:
Charlotte_merged.loc[Charlotte_merged['Cluster Labels'] == 2, Charlotte_merged.columns[[1] + list(range(5, Charlotte_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
285,35.250841,-80.812530,Brewery,2,American Restaurant,Food Truck,Burger Joint,Fast Food Restaurant,Brewery,Grocery Store,Discount Store,Farmers Market,Dry Cleaner,Donut Shop
286,35.250841,-80.816149,Food Truck,2,American Restaurant,Food Truck,Burger Joint,Fast Food Restaurant,Brewery,Grocery Store,Discount Store,Farmers Market,Dry Cleaner,Donut Shop
287,35.250841,-80.811473,Burger Joint,2,American Restaurant,Food Truck,Burger Joint,Fast Food Restaurant,Brewery,Grocery Store,Discount Store,Farmers Market,Dry Cleaner,Donut Shop
288,35.250841,-80.827034,American Restaurant,2,American Restaurant,Food Truck,Burger Joint,Fast Food Restaurant,Brewery,Grocery Store,Discount Store,Farmers Market,Dry Cleaner,Donut Shop
289,35.250841,-80.817952,Fast Food Restaurant,2,American Restaurant,Food Truck,Burger Joint,Fast Food Restaurant,Brewery,Grocery Store,Discount Store,Farmers Market,Dry Cleaner,Donut Shop
290,35.250841,-80.829505,Discount Store,2,American Restaurant,Food Truck,Burger Joint,Fast Food Restaurant,Brewery,Grocery Store,Discount Store,Farmers Market,Dry Cleaner,Donut Shop
291,35.250841,-80.826131,American Restaurant,2,American Restaurant,Food Truck,Burger Joint,Fast Food Restaurant,Brewery,Grocery Store,Discount Store,Farmers Market,Dry Cleaner,Donut Shop
292,35.250841,-80.829505,Grocery Store,2,American Restaurant,Food Truck,Burger Joint,Fast Food Restaurant,Brewery,Grocery Store,Discount Store,Farmers Market,Dry Cleaner,Donut Shop


In [70]:
Charlotte_merged.loc[Charlotte_merged['Cluster Labels'] == 3, Charlotte_merged.columns[[1] + list(range(5, Charlotte_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,35.26002,-80.797171,Fried Chicken Joint,3,Health & Beauty Service,Seafood Restaurant,Post Office,Rental Car Location,Grocery Store,Art Gallery,Spanish Restaurant,Convenience Store,Bus Station,Fried Chicken Joint
1,35.26002,-80.807570,Rental Car Location,3,Health & Beauty Service,Seafood Restaurant,Post Office,Rental Car Location,Grocery Store,Art Gallery,Spanish Restaurant,Convenience Store,Bus Station,Fried Chicken Joint
2,35.26002,-80.801427,Convenience Store,3,Health & Beauty Service,Seafood Restaurant,Post Office,Rental Car Location,Grocery Store,Art Gallery,Spanish Restaurant,Convenience Store,Bus Station,Fried Chicken Joint
3,35.26002,-80.801192,Post Office,3,Health & Beauty Service,Seafood Restaurant,Post Office,Rental Car Location,Grocery Store,Art Gallery,Spanish Restaurant,Convenience Store,Bus Station,Fried Chicken Joint
4,35.26002,-80.793654,Grocery Store,3,Health & Beauty Service,Seafood Restaurant,Post Office,Rental Car Location,Grocery Store,Art Gallery,Spanish Restaurant,Convenience Store,Bus Station,Fried Chicken Joint
5,35.26002,-80.807855,Art Gallery,3,Health & Beauty Service,Seafood Restaurant,Post Office,Rental Car Location,Grocery Store,Art Gallery,Spanish Restaurant,Convenience Store,Bus Station,Fried Chicken Joint
6,35.26002,-80.796583,Bank,3,Health & Beauty Service,Seafood Restaurant,Post Office,Rental Car Location,Grocery Store,Art Gallery,Spanish Restaurant,Convenience Store,Bus Station,Fried Chicken Joint
7,35.26002,-80.797620,Health & Beauty Service,3,Health & Beauty Service,Seafood Restaurant,Post Office,Rental Car Location,Grocery Store,Art Gallery,Spanish Restaurant,Convenience Store,Bus Station,Fried Chicken Joint
8,35.26002,-80.804712,Seafood Restaurant,3,Health & Beauty Service,Seafood Restaurant,Post Office,Rental Car Location,Grocery Store,Art Gallery,Spanish Restaurant,Convenience Store,Bus Station,Fried Chicken Joint
9,35.26002,-80.794499,Spanish Restaurant,3,Health & Beauty Service,Seafood Restaurant,Post Office,Rental Car Location,Grocery Store,Art Gallery,Spanish Restaurant,Convenience Store,Bus Station,Fried Chicken Joint


In [71]:
Charlotte_merged.loc[Charlotte_merged['Cluster Labels'] == 4, Charlotte_merged.columns[[1] + list(range(5, Charlotte_merged.shape[1]))]]

,Neighborhood Latitude,Venue Longitude,Venue Category,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
293,35.195393,-80.826092,American Restaurant,4,Italian Restaurant,American Restaurant,Spa,French Restaurant,Intersection,Coffee Shop,Gym / Fitness Center,Pharmacy,Salon / Barbershop,Mexican Restaurant
294,35.195393,-80.826598,Supermarket,4,Italian Restaurant,American Restaurant,Spa,French Restaurant,Intersection,Coffee Shop,Gym / Fitness Center,Pharmacy,Salon / Barbershop,Mexican Restaurant
295,35.195393,-80.824654,Gym / Fitness Center,4,Italian Restaurant,American Restaurant,Spa,French Restaurant,Intersection,Coffee Shop,Gym / Fitness Center,Pharmacy,Salon / Barbershop,Mexican Restaurant
296,35.195393,-80.825357,Asian Restaurant,4,Italian Restaurant,American Restaurant,Spa,French Restaurant,Intersection,Coffee Shop,Gym / Fitness Center,Pharmacy,Salon / Barbershop,Mexican Restaurant
297,35.195393,-80.824495,Mexican Restaurant,4,Italian Restaurant,American Restaurant,Spa,French Restaurant,Intersection,Coffee Shop,Gym / Fitness Center,Pharmacy,Salon / Barbershop,Mexican Restaurant
298,35.195393,-80.827493,Bed & Breakfast,4,Italian Restaurant,American Restaurant,Spa,French Restaurant,Intersection,Coffee Shop,Gym / Fitness Center,Pharmacy,Salon / Barbershop,Mexican Restaurant
299,35.195393,-80.824301,Wine Bar,4,Italian Restaurant,American Restaurant,Spa,French Restaurant,Intersection,Coffee Shop,Gym / Fitness Center,Pharmacy,Salon / Barbershop,Mexican Restaurant
300,35.195393,-80.824693,Movie Theater,4,Italian Restaurant,American Restaurant,Spa,French Restaurant,Intersection,Coffee Shop,Gym / Fitness Center,Pharmacy,Salon / Barbershop,Mexican Restaurant
301,35.195393,-80.825530,Italian Restaurant,4,Italian Restaurant,American Restaurant,Spa,French Restaurant,Intersection,Coffee Shop,Gym / Fitness Center,Pharmacy,Salon / Barbershop,Mexican Restaurant
302,35.195393,-80.824363,Gym / Fitness Center,4,Italian Restaurant,American Restaurant,Spa,French Restaurant,Intersection,Coffee Shop,Gym / Fitness Center,Pharmacy,Salon / Barbershop,Mexican Restaurant
